# Chapter 8: Dimensionality Reduction

Many machine learning problems involve thousands or even millions of
features for each training instance. Not only do all these features make
training extremely slow, but they can also make it much harder to find a good
solution, as you will see.

Apart from speeding up training, dimensionality reduction is also extremely
useful for data visualization. Reducing the number of dimensions down to
two (or three) makes it possible to plot a condensed view of a 
high-dimensional training set on a graph and often gain some important insights by
visually detecting patterns, such as clusters. Moreover, data visualization is
essential to communicate your conclusions to people who are not data
scientists—in particular, decision makers who will use your results.

## The Curse of Dimensionality

We are so used to living in three dimensions that our intuition fails us
when we try to imagine a high-dimensional space. It turns out that many
things behave very differently in high-dimensional space.

Here is a more troublesome difference: if you pick two points randomly in a
unit square, the distance between these two points will be, on average,
roughly 0.52. If you pick two random points in a 3D unit cube, the average
distance will be roughly 0.66. But what about two points picked randomly in
a 1,000,000-dimensional unit hypercube? The average distance, believe it or
not, will be about 408.25 (roughly 1,000,0006)! This is counterintuitive: how
can two points be so far apart when they both lie within the same unit
hypercube? Well, there’s just plenty of space in high dimensions. As a result,
high-dimensional datasets are at risk of being very sparse: most training
instances are likely to be far away from each other. This also means that a
new instance will likely be far away from any training instance, making
predictions much less reliable than in lower dimensions, since they will be
based on much larger extrapolations. In short, the more dimensions the
training set has, the greater the risk of overfitting it.


## Main Approaches for Dimensionality Reduction

Before we dive into specific dimensionality reduction algorithms, let’s take a
look at the two main approaches to reducing dimensionality: projection and
manifold learning.

### Projection

In most real-world problems, training instances are not spread out uniformly
across all dimensions. Many features are almost constant, while others are
highly correlated (as discussed earlier for MNIST). As a result, all training
instances lie within (or close to) a much lower-dimensional subspace of the
high-dimensional space.

### Manifold Learning

However, projection is not always the best approach to dimensionality
reduction. In many cases the subspace may twist and turn, such as in the
famous Swiss roll toy dataset.

Many dimensionality reduction algorithms work by modeling the manifold
on which the training instances lie; this is called manifold learning. It relies
on the manifold assumption, also called the manifold hypothesis, which holds
that most real-world high-dimensional datasets lie close to a much 
lower-dimensional manifold. This assumption is very often empirically observed.

In short, reducing the dimensionality of your training set before training a
model will usually speed up training, but it may not always lead to a better or
simpler solution; it all depends on the dataset.


## PCA
Principal component analysis (PCA) is by far the most popular
dimensionality reduction algorithm. First it identifies the hyperplane that lies
closest to the data, and then it projects the data onto it.

### Preserving the Variance
It seems reasonable to select the axis that preserves the maximum amount of
variance, as it will most likely lose less information than the other
projections. Another way to justify this choice is that it is the axis that
minimizes the mean squared distance between the original dataset and its
projection onto that axis. This is the rather simple idea behind PCA.

### Pricipal Components
PCA identifies the axis that accounts for the largest amount of variance in the
training set. It also finds a second axis,
orthogonal to the first one, that accounts for the largest amount of the
remaining variance. In this 2D example there is no choice: it is the dotted
line. If it were a higher-dimensional dataset, PCA would also find a third
axis, orthogonal to both previous axes, and a fourth, a fifth, and so on—as
many axes as the number of dimensions in the dataset.

The $i^th$ axis is called the $i^th$ principal component (PC) of the data.

So how can you find the principal components of a training set? Luckily,
there is a standard matrix factorization technique called singular value
decomposition (SVD) that can decompose the training set matrix X into the
matrix multiplication of three matrices U Σ V , where V contains the unit
vectors that define all the principal components that you are looking for.

### Projecting Down to d Dimensions
Once you have identified all the principal components, you can reduce the
dimensionality of the dataset down to d dimensions by projecting it onto the
hyperplane defined by the first d principal components. Selecting this
hyperplane ensures that the projection will preserve as much variance as
possible.

### Explained Variance Ratio
Another useful piece of information is the **explained variance ratio** of each
principal component, available via the explained_variance_ratio_ variable.
The ratio indicates the *proportion of the dataset’s variance* that lies along
each principal component.

### Choosing the Right Number of Dimensions
Instead of arbitrarily choosing the number of dimensions to reduce down to,
it is simpler to choose the number of dimensions that add up to a sufficiently
large portion of the variance—say, 95% (An exception to this rule, of course,
is if you are reducing dimensionality for data visualization, in which case you
will want to reduce the dimensionality down to 2 or 3).

### PCA for Compression
After dimensionality reduction, the training set takes up much less space. For
example, after applying PCA to the MNIST dataset while preserving 95% of
its variance, we are left with 154 features, instead of the original 784 features.
So the dataset is now less than 20% of its original size, and we only lost 5%
of its variance! This is a reasonable compression ratio, and it’s easy to see
how such a size reduction would speed up a classification algorithm
tremendously.
It is also possible to decompress the reduced dataset back to 784 dimensions
by applying the inverse transformation of the PCA projection. This won’t
give you back the original data, since the projection lost a bit of information
(within the 5% variance that was dropped), but it will likely be close to the
original data. The mean squared distance between the original data and the
reconstructed data (compressed and then decompressed) is called the
reconstruction error.

### Randomized PCA
If you set the svd_solver hyperparameter to "randomized", Scikit-Learn uses
a stochastic algorithm called randomized PCA that quickly finds an
approximation of the first d principal components. Its computational
complexity is O(m × d ) + O(d ), instead of O(m × n ) + O(n ) for the full
SVD approach, so it is dramatically faster than full SVD when d is much
smaller than n.

### Incremental PCA
One problem with the preceding implementations of PCA is that they require
the whole training set to fit in memory in order for the algorithm to run.
Fortunately, incremental PCA (IPCA) algorithms have been developed that
allow you to split the training set into mini-batches and feed these in one
mini-batch at a time. This is useful for large training sets and for applying
PCA online (i.e., on the fly, as new instances arrive).

## Random Projection
As its name suggests, the random projection algorithm projects the data to a
lower-dimensional space using a random linear projection. This may sound
crazy, but it turns out that such a random projection is actually very likely to
preserve distances fairly well, as was demonstrated mathematically by
William B. Johnson and Joram Lindenstrauss in a famous lemma. So, two
similar instances will remain similar after the projection, and two very
different instances will remain very different.

Obviously, the more dimensions you drop, the more information is lost, and
the more distances get distorted. So how can you choose the optimal number
of dimensions? Well, Johnson and Lindenstrauss came up with an equation
that determines the minimum number of dimensions to preserve in order to
ensure—with high probability—that distances won’t change by more than a
given tolerance. For example, if you have a dataset containing m = 5,000
instances with n = 20,000 features each, and you don’t want the squared
distance between any two instances to change by more than ε = 10%, then
you should project the data down to d dimensions, with d ≥ 4 log(m) / (½ ε² -
⅓ ε³), which is 7,300 dimensions. That’s quite a significant dimensionality
reduction! Notice that the equation does not use n, it only relies on m and ε.

In summary, random projection is a simple, fast, memory-efficient, and
surprisingly powerful dimensionality reduction algorithm that you should
keep in mind, especially when you deal with high-dimensional datasets.

## LLE
Locally linear embedding (LLE) is a nonlinear dimensionality reduction
(NLDR) technique. It is a manifold learning technique that does not rely on
projections, unlike PCA and random projection. In a nutshell, LLE works by
first measuring how each training instance linearly relates to its nearest
neighbors, and then looking for a low-dimensional representation of the
training set where these local relationships are best preserved (more details
shortly). This approach makes it particularly good at unrolling twisted
manifolds, especially when there is not too much noise.

Here’s how LLE works: for each training instance x , the algorithm
identifies its k-nearest neighbors (in the preceding code k = 10), then tries to
reconstruct x as a linear function of these neighbors.

After this step, the weight matrix W ^ (containing the weights w^i,j) encodes
the local linear relationships between the training instances. The second step
is to map the training instances into a d-dimensional space (where d < n)
while preserving these local relationships as much as possible.

As you can see, LLE is quite different from the projection techniques, and it’s
significantly more complex, but it can also construct much better 
low-dimensional representations, especially if the data is nonlinear.